# Part 1

In [11]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#get your URL
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
#Scraping your URL
soup = BeautifulSoup(website_text,'xml')

In [4]:
#Define your table from your wiki
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
print(table_rows)

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [5]:
#Create a loop to organize your data
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
print(data)

[[], ['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Islington'], ['M9B', 'Etobicoke', 'Martin

In [6]:
#Create your dataframe
df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows, if you skip it you'll have rows with 'none'
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [7]:
#Drop rows with Borough 'not assigned'
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace= True)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [34]:
# More than one neighborhood can exist in one postal code area. We have to combine them into one row with the neighborhoods separated with a comma.
df_postal=df.groupby("PostalCode").agg(lambda x:','.join(set(x))).reset_index() #add the reset index to be able to merge the two tables (having the same shape)
df_postal.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"West Hill,Guildwood,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [35]:
#Show the shape
df_postal.shape

(103, 3)

# Part 2

In [33]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [63]:
#We have to rename the column because, we will join our 2 tables with PostalCode (There is a space between words)
df_geo.rename(columns={'Postal Code':'PostalCode'}, 
                 inplace=True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
#merge the table with key column (PostalCode)
df_merge = df_postal.set_index('PostalCode').join(df_geo.set_index('PostalCode')).reset_index()
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [43]:
df_merge.shape

(103, 5)

In [44]:
#To build my analysis, I choose Toronto
df_toronto = df_merge[df_merge['Borough'].str.contains('Toronto')]
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Summerhill East,Moore Park",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West,South Hill,Forest Hill SE,Rath...",43.686412,-79.400049


# Clustering Toronto

### Libraries

In [64]:
#Import libraries like in NYC

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Define Your Coordinate 

In [65]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a Map 

In [68]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define your clusters 

In [81]:
#Select the colors of your clusters
Cluster_colors = ['darkred', 'darkgreen', 'gold', 'darkblue']

#Select randomly a number of cluster (as we can see, there are approximatly 4 clusters on the map)
n_clusters = 4

# do a k-means clustering with k=4
kmean_classifier = KMeans(n_clusters=n_clusters)

In [83]:
# Create a new dataframe with just the geodata to plot on the map
X_train = df_toronto[['Latitude', 'Longitude']]
X_train.head()

,Latitude,Longitude
37,43.676357,-79.293031
41,43.679557,-79.352188
42,43.668999,-79.315572
43,43.659526,-79.340923
44,43.728020,-79.388790


In [84]:
#Train your model to find the centers of your clusters
kmean_classifier.fit(X_train)
kmean_classifier.cluster_centers_

array([[ 43.70563855, -79.39811351],
       [ 43.65506566, -79.44547176],
       [ 43.66943648, -79.32465436],
       [ 43.65434514, -79.38272671]])

### Create a Map with your clusters

In [94]:
toronto_coord = (43.653963, -79.387207)
toronto_map = folium.Map(location=toronto_coord, zoom_start=12)

# add clustered markers to the map
for idx, lat, lng, borough, neighborhood in zip(range(df_toronto.shape[0]), df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    color_idx = kmean_classifier.labels_[idx]
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    #Increasing your radius, make your points bigger
    folium.CircleMarker([lat, lng], radius=8, popup=label, color=fill_colors[color_idx],\
        fill=True, fill_color=fill_colors[color_idx], fill_opacity=10.0, parse_html=False)\
        .add_to(toronto_map)

# add cluster center markers to the map
for idx, (lat, lng) in enumerate(kmean_classifier.cluster_centers_):
    color_idx = idx
    label = 'cluster #{}'.format(idx + 1)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=8, popup=label, color=cluster_fill_colors[color_idx],\
        fill=True, fill_color='white', fill_opacity=10.0, parse_html=False)\
        .add_to(toronto_map)

toronto_map